In [512]:
import numpy as np
import os
import re
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import keras
from keras.utils import to_categorical
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

In [589]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [590]:
dirname = os.path.join(os.getcwd(), '/content/drive/MyDrive/UNIVERSIDAD_CENTRAL/2021-1S/INTELIGENCIA_ARTIFICIAL/PRUEBA')
imgpath = dirname + os.sep 
print(imgpath)

images = []
directories = []
dircount = []
prevRoot=''
cant=0

print("leyendo imagenes de ",imgpath)

for root, dirnames, filenames in os.walk(imgpath):
    for filename in filenames:
        if re.search("\.(jpg|jpeg|png|bmp|tiff)$", filename):
            cant=cant+1
            filepath = os.path.join(root, filename)
            
            image = plt.imread(filepath)
            images.append(image)
            b = "Leyendo..." + str(cant)

            print (b, end="\r")
            if prevRoot !=root:
                print(root, cant)
                prevRoot=root
                directories.append(root)
                dircount.append(cant)
                cant=0
dircount.append(cant)
dircount = dircount[1:]
dircount[0]=dircount[0]+1
print('Directorios leidos:',len(directories))
print("Imagenes en cada directorio", dircount)
print('suma Total de imagenes en subdirs:',sum(dircount))

/content/drive/MyDrive/UNIVERSIDAD_CENTRAL/2021-1S/INTELIGENCIA_ARTIFICIAL/PRUEBA/
leyendo imagenes de  /content/drive/MyDrive/UNIVERSIDAD_CENTRAL/2021-1S/INTELIGENCIA_ARTIFICIAL/PRUEBA/
/content/drive/MyDrive/UNIVERSIDAD_CENTRAL/2021-1S/INTELIGENCIA_ARTIFICIAL/PRUEBA/piel 1
/content/drive/MyDrive/UNIVERSIDAD_CENTRAL/2021-1S/INTELIGENCIA_ARTIFICIAL/PRUEBA/hundiment 48
/content/drive/MyDrive/UNIVERSIDAD_CENTRAL/2021-1S/INTELIGENCIA_ARTIFICIAL/PRUEBA/bach 48
Directorios leidos: 3
Imagenes en cada directorio [49, 48, 56]
suma Total de imagenes en subdirs: 153


In [591]:
labels=[]
indice=0
for cantidad in dircount:
    for i in range(cantidad):
        labels.append(indice)
    indice=indice+1
print("Cantidad etiquetas creadas: ",len(labels))
 
deportes=[]
indice=0
for directorio in directories:
    name = directorio.split(os.sep)
    print(indice , name[len(name)-1])
    deportes.append(name[len(name)-1])
    indice=indice+1
 
y = np.array(labels)
X = np.array(images, dtype=np.uint8)
print(X)

# Find the unique numbers from the train labels
classes = np.unique(y)
nClasses = len(classes)
print('Total number of outputs : ', nClasses)
print('Output classes : ', classes)

Cantidad etiquetas creadas:  153
0 piel
1 hundiment
2 bach
[[[[135 135 135]
   [126 126 126]
   [133 133 133]
   ...
   [133 133 133]
   [137 137 137]
   [139 139 139]]

  [[130 130 130]
   [142 142 142]
   [126 126 126]
   ...
   [134 134 134]
   [138 138 138]
   [143 143 143]]

  [[122 122 122]
   [139 139 139]
   [125 125 125]
   ...
   [132 132 132]
   [133 133 133]
   [138 138 138]]

  ...

  [[126 126 126]
   [121 121 121]
   [127 127 127]
   ...
   [133 133 133]
   [133 133 133]
   [136 136 136]]

  [[125 125 125]
   [125 125 125]
   [125 125 125]
   ...
   [131 131 131]
   [131 131 131]
   [134 134 134]]

  [[130 130 130]
   [135 135 135]
   [123 123 123]
   ...
   [121 121 121]
   [121 121 121]
   [126 126 126]]]


 [[[146 212 255]
   [145 213 250]
   [152 206 250]
   ...
   [ 60  69  40]
   [ 50  61  45]
   [ 57  70  52]]

  [[115 151 183]
   [ 83 126 133]
   [106 138 163]
   ...
   [ 65  69  54]
   [ 55  59  60]
   [ 57  66  65]]

  [[ 39  45  45]
   [ 38  46  31]
   [ 48  5

In [599]:
#Mezclar todo y crear los grupos de entrenamiento y testing
train_X,test_X,train_Y,test_Y = train_test_split(X,y,test_size=0.2)
print('Training data shape : ', train_X.shape, train_Y.shape)
print('Testing data shape : ', test_X.shape, test_Y.shape)
 
train_X = train_X.astype(np.float32)
test_X = test_X.astype(np.float32)
train_X = train_X / 255.
test_X = test_X / 255.
 
# Change the labels from categorical to one-hot encoding
train_Y_one_hot = to_categorical(train_Y)
test_Y_one_hot = to_categorical(test_Y)
 
# Display the change for category label using one-hot encoding
print('Original label:', train_Y[0])
print('After conversion to one-hot:', train_Y_one_hot[0])
 
train_X,valid_X,train_label,valid_label = train_test_split(train_X, train_Y_one_hot, test_size=0.2, random_state=13)
 
print(train_X.shape,valid_X.shape,train_label.shape,valid_label.shape)

Training data shape :  (122, 28, 21, 3) (122,)
Testing data shape :  (31, 28, 21, 3) (31,)
Original label: 2
After conversion to one-hot: [0. 0. 1.]
(97, 28, 21, 3) (25, 28, 21, 3) (97, 3) (25, 3)


In [600]:
INIT_LR = 1e-3
epochs = 9
batch_size = 64
 
gap_model = Sequential()
gap_model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',padding='same',input_shape=(21,28,3)))
gap_model.add(LeakyReLU(alpha=0.1))
gap_model.add(MaxPooling2D((2, 2),padding='same'))
gap_model.add(Dropout(0.5))
 
gap_model.add(Flatten())
gap_model.add(Dense(32, activation='linear'))
gap_model.add(LeakyReLU(alpha=0.1))
gap_model.add(Dropout(0.5)) 
gap_model.add(Dense(nClasses, activation='softmax'))
 
gap_model.summary()
 
gap_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adagrad(lr=INIT_LR, decay=INIT_LR / 100),metrics=['accuracy'])


Model: "sequential_74"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_74 (Conv2D)           (None, 21, 28, 32)        896       
_________________________________________________________________
leaky_re_lu_148 (LeakyReLU)  (None, 21, 28, 32)        0         
_________________________________________________________________
max_pooling2d_74 (MaxPooling (None, 11, 14, 32)        0         
_________________________________________________________________
dropout_148 (Dropout)        (None, 11, 14, 32)        0         
_________________________________________________________________
flatten_74 (Flatten)         (None, 4928)              0         
_________________________________________________________________
dense_148 (Dense)            (None, 32)                157728    
_________________________________________________________________
leaky_re_lu_149 (LeakyReLU)  (None, 32)              

In [602]:
gap_train_dropout = gap_model.fit(train_X, train_label, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_X, valid_label))
 
#guardamos la red, para reutilizarla en el futuro, sin tener que volver a entrenar
gap_model.save("gap_mnist.h5py")

Epoch 1/9
2/2 [==============================] - 0s 110ms/step - loss: 1.0680 - accuracy: 0.4536 - val_loss: 1.1154 - val_accuracy: 0.3600
Epoch 2/9
2/2 [==============================] - 0s 63ms/step - loss: 1.1268 - accuracy: 0.3299 - val_loss: 1.1162 - val_accuracy: 0.2400
Epoch 3/9
2/2 [==============================] - 0s 63ms/step - loss: 1.1511 - accuracy: 0.2990 - val_loss: 1.1169 - val_accuracy: 0.2000
Epoch 4/9
2/2 [==============================] - 0s 67ms/step - loss: 1.0952 - accuracy: 0.4124 - val_loss: 1.1143 - val_accuracy: 0.2800
Epoch 5/9
2/2 [==============================] - 0s 68ms/step - loss: 1.1224 - accuracy: 0.3711 - val_loss: 1.1162 - val_accuracy: 0.3200
Epoch 6/9
2/2 [==============================] - 0s 60ms/step - loss: 1.1715 - accuracy: 0.2784 - val_loss: 1.1170 - val_accuracy: 0.2000
Epoch 7/9
2/2 [==============================] - 0s 69ms/step - loss: 1.1609 - accuracy: 0.2990 - val_loss: 1.1195 - val_accuracy: 0.2000
Epoch 8/9
2/2 [==================